In [1]:
import ee
ee.Initialize()

In [2]:
from geebap import scores

In [6]:
from geetools import tools, cloud_mask, algorithms, collection

In [7]:
import ipygee as ui

In [8]:
Map = ui.Map()

In [9]:
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [10]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

Masked image

In [11]:
i = ee.Image('COPERNICUS/S2/20181122T142749_20181122T143353_T18GYT')

In [12]:
s2col = collection.Sentinel2TOA()

In [13]:
s2col.bitOptions()

{'QA60': ['cloud', 'cirrus']}

In [14]:
masked = s2col.applyMask(i, 'QA60', ['cloud'])
masked = masked.clip(aoi)

In [15]:
vis = {'bands':['B8', 'B11', 'B4'], 'min':0, 'max':5000}

In [16]:
Map.addLayer(masked, vis, 'Masked')

In [17]:
Map.centerObject(masked)

## CloudDist score

In [18]:
cld = scores.CloudDist()

In [19]:
cldband = cld.compute(masked, bandmask='B2', dmin=0, dmax=600)

In [20]:
Map.addLayer(cldband, {'min':0, 'max':1, 'palette':['white', 'red', 'orange', 'yellow', 'green', 'blue', 'black']}, 'Cloud Distance Score')

## Using collection module

In [23]:
masked = s2col.rename(masked)

In [24]:
icol = ee.ImageCollection.fromImages([masked])

In [25]:
col_score = cld.map(icol, col=s2col)

In [26]:
newi = ee.Image(col_score.first())

In [27]:
Map.addLayer(newi, {'bands':[cld.name], 'min':0, 'max':1}, 'Cloud Distance Score from satcol')